In [1]:
import numpy as np
import pandas as pd
import warnings
import itertools
from tqdm import tqdm_notebook
from scipy.stats import spearmanr
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv(f'data/train.csv')
TARGETS = train.columns[11:]

for col in TARGETS:
    train[col] = train[col].rank(method="average")
train[TARGETS] = MinMaxScaler().fit_transform(train[TARGETS])
y = train[TARGETS].values
ix = np.where(train.groupby("question_body")["host"].transform("count") == 1)[0]

y_use = pd.read_csv('oofs/use_oof.csv').values
y_albert = pd.read_csv('oofs/siamese_albert_1_comb_oofs.csv', index_col=0).values
y_bert = pd.read_csv('oofs/siamese_bert_6_oofs.csv', index_col=0).values
y_roberta = pd.read_csv('oofs/siamese_roberta_1_comb_oofs.csv', index_col=0).values
# y_xlnet = pd.read_csv('oofs/siamese_xlnet_1_comb_oofs.csv', index_col=0).values
y_xlnet = pd.read_csv('oofs/siamese_xlnet_2_comb_oofs.csv', index_col=0).values

preds = [y_use, y_roberta, y_bert, y_xlnet, y_albert]
n_models = len(preds)

In [3]:
def bce(y, y_pred):
    return -np.mean(y*np.log(y_pred + 1e-5) + (1-y)*np.log(1-y_pred + 1e-5))


def evaluate(y, y_pred, verbose=False, exclude=[]):
    score = 0
    for i in range(y.shape[1]):
        if i not in exclude:
            col_score = spearmanr(y[:, i], y_pred[:, i])[0]
            if verbose:
                print(TARGETS[i], np.round(col_score, 3))
            score += col_score/(y.shape[1]-len(exclude))
    return np.round(score, 4)


ds = [4, 8, 16, 32, 64, None]
ws = [0, 1, 2, 4]


def scale(x, d):
    if d:
        return (x//(1/d))/d
    return x

dws5 = list(itertools.product(ds, ds, *(n_models*[ws])))


def transform(preds, params, c):
    (d_global, d_local), ws = params[:2], params[2:]
    y_temp = 0
    for pred, w in zip(preds, ws):
        y_temp += w * scale(pred[:, c], d_local)
    y_temp /= sum(ws)
    y_temp = scale(y_temp, d_global)
    return y_temp


bce_weight = 0.25


def f(c):
    max_score = spearmanr(y[:, c], y_roberta[:, c])[0] - bce_weight * bce(y[:, c], y_roberta[:, c])
    best_index = -1
    for i, params in enumerate(dws5):
        if sum(params[2:]) == 0:
            continue
        y_temp = transform(preds, params, c)
        score = spearmanr(y[:, c], y_temp)[0] - bce_weight * bce(y[:, c], y_temp)
        if score > max_score:
            max_score = score
            best_index = i
            
    return best_index

In [4]:
import multiprocessing

pool = multiprocessing.Pool(15)
out = pool.map(f, range(30))

In [5]:
y_combined = np.zeros(y.shape)

for c in range(30):
    print(c, TARGETS[c], dws5[out[c]])
    y_combined[:, c] = transform(preds, dws5[out[c]], c)

0 question_asker_intent_understanding (None, None, 2, 2, 1, 2, 2)
1 question_body_critical (None, None, 2, 0, 1, 0, 1)
2 question_conversational (16, 32, 1, 0, 1, 1, 0)
3 question_expect_short_answer (32, 16, 2, 4, 2, 1, 2)
4 question_fact_seeking (16, 32, 2, 1, 2, 4, 1)
5 question_has_commonly_accepted_answer (4, None, 1, 4, 2, 1, 4)
6 question_interestingness_others (32, None, 4, 1, 2, 2, 4)
7 question_interestingness_self (16, None, 2, 1, 1, 2, 1)
8 question_multi_intent (64, 64, 4, 1, 2, 4, 4)
9 question_not_really_a_question (32, 32, 1, 2, 4, 1, 2)
10 question_opinion_seeking (64, None, 2, 4, 1, 4, 2)
11 question_type_choice (64, None, 1, 2, 1, 1, 2)
12 question_type_compare (8, 8, 2, 4, 2, 4, 1)
13 question_type_consequence (32, 8, 0, 1, 0, 4, 0)
14 question_type_definition (8, 8, 0, 4, 1, 0, 0)
15 question_type_entity (8, 32, 2, 2, 0, 4, 1)
16 question_type_instructions (None, None, 1, 1, 0, 1, 1)
17 question_type_procedure (64, None, 1, 4, 0, 2, 1)
18 question_type_reason_expla

In [6]:
evaluate(y, y_combined, True)

question_asker_intent_understanding 0.404
question_body_critical 0.691
question_conversational 0.489
question_expect_short_answer 0.326
question_fact_seeking 0.382
question_has_commonly_accepted_answer 0.486
question_interestingness_others 0.374
question_interestingness_self 0.527
question_multi_intent 0.611
question_not_really_a_question 0.171
question_opinion_seeking 0.497
question_type_choice 0.759
question_type_compare 0.568
question_type_consequence 0.323
question_type_definition 0.669
question_type_entity 0.61
question_type_instructions 0.791
question_type_procedure 0.385
question_type_reason_explanation 0.688
question_type_spelling 0.608
question_well_written 0.542
answer_helpful 0.287
answer_level_of_information 0.48
answer_plausible 0.188
answer_relevance 0.199
answer_satisfaction 0.389
answer_type_instructions 0.773
answer_type_procedure 0.329
answer_type_reason_explanation 0.703
answer_well_written 0.252


0.4833

In [7]:
evaluate(y, y_combined, True, exclude=[9,19])

question_asker_intent_understanding 0.404
question_body_critical 0.691
question_conversational 0.489
question_expect_short_answer 0.326
question_fact_seeking 0.382
question_has_commonly_accepted_answer 0.486
question_interestingness_others 0.374
question_interestingness_self 0.527
question_multi_intent 0.611
question_opinion_seeking 0.497
question_type_choice 0.759
question_type_compare 0.568
question_type_consequence 0.323
question_type_definition 0.669
question_type_entity 0.61
question_type_instructions 0.791
question_type_procedure 0.385
question_type_reason_explanation 0.688
question_well_written 0.542
answer_helpful 0.287
answer_level_of_information 0.48
answer_plausible 0.188
answer_relevance 0.199
answer_satisfaction 0.389
answer_type_instructions 0.773
answer_type_procedure 0.329
answer_type_reason_explanation 0.703
answer_well_written 0.252


0.49

In [8]:
[dws5[out[c]] for c in range(30)]

[(None, None, 2, 2, 1, 2, 2),
 (None, None, 2, 0, 1, 0, 1),
 (16, 32, 1, 0, 1, 1, 0),
 (32, 16, 2, 4, 2, 1, 2),
 (16, 32, 2, 1, 2, 4, 1),
 (4, None, 1, 4, 2, 1, 4),
 (32, None, 4, 1, 2, 2, 4),
 (16, None, 2, 1, 1, 2, 1),
 (64, 64, 4, 1, 2, 4, 4),
 (32, 32, 1, 2, 4, 1, 2),
 (64, None, 2, 4, 1, 4, 2),
 (64, None, 1, 2, 1, 1, 2),
 (8, 8, 2, 4, 2, 4, 1),
 (32, 8, 0, 1, 0, 4, 0),
 (8, 8, 0, 4, 1, 0, 0),
 (8, 32, 2, 2, 0, 4, 1),
 (None, None, 1, 1, 0, 1, 1),
 (64, None, 1, 4, 0, 2, 1),
 (32, None, 2, 2, 1, 2, 1),
 (16, 64, 2, 1, 4, 0, 0),
 (64, None, 2, 1, 4, 2, 0),
 (32, 64, 1, 4, 0, 0, 4),
 (64, None, 4, 4, 1, 2, 2),
 (32, None, 1, 2, 0, 0, 2),
 (16, 16, 1, 2, 2, 2, 1),
 (64, None, 1, 2, 0, 1, 2),
 (None, 64, 2, 4, 1, 2, 2),
 (64, None, 2, 4, 2, 1, 1),
 (32, None, 4, 4, 1, 2, 4),
 (32, None, 0, 1, 4, 4, 2)]